In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
x=pd.read_csv('../input/train.tsv',delimiter='\t')
y=pd.read_csv('../input/test.tsv',delimiter='\t')

In [ ]:
import re
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score,confusion_matrix
from keras.preprocessing import sequence,text
from keras.layers import LSTM,Embedding,Dropout,Conv1D,GlobalMaxPooling1D,Dense
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

In [ ]:
from nltk.stem import WordNetLemmatizer,SnowballStemmer
net=WordNetLemmatizer()
snow=SnowballStemmer('english')
def cleaner(xxx):
    xxx=xxx.lower()
    xxx=re.sub('[^a-zA-Z0-9\s]','',xxx)
    xx=[net.lemmatize(y) for y in xxx.split(" ")]
    xx=' '.join(xx)
    return xx
x['cleaned']=x['Phrase'].map(cleaner)
y['cleaned']=y['Phrase'].map(cleaner)
    
    

In [ ]:
z=to_categorical(x['Sentiment'],num_classes=5)

In [ ]:
xtrain,xtest,ztrain,ztest=tts(x['cleaned'],z,train_size=0.7)

In [ ]:
from keras.preprocessing.text import Tokenizer
t=Tokenizer()

In [ ]:
t.fit_on_texts(list(xtrain))

In [ ]:
xtrain=t.texts_to_sequences(xtrain)
xtest=t.texts_to_sequences(xtest)
test=t.texts_to_sequences(y['cleaned'])

In [ ]:
xtrain=sequence.pad_sequences(xtrain,maxlen=52)
xtest=sequence.pad_sequences(xtest,maxlen=52)
test=sequence.pad_sequences(test,maxlen=52)

In [ ]:
model=Sequential()
model.add(Embedding(15049,512,input_length=52))
model.add(LSTM(254,dropout=0.1,recurrent_dropout=0.1,return_sequences=True))
model.add(LSTM(128,dropout=0.1,recurrent_dropout=0.1,return_sequences=False))
model.add(Dense(5,activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='RMSProp',metrics=['accuracy'])

In [ ]:
model.fit(xtrain,ztrain,validation_data=(xtest,ztest),epochs=10,batch_size=128,verbose=1)

In [ ]:
pf=model.predict_classes(test)
pff=pd.DataFrame(pf,columns=['Sentiment'],index=y['PhraseId'])
pff.index.name='PhraseId'
pff

In [ ]:
pff.to_csv('gotit.csv')